In [1]:
#pip install keras
#pip install tensorflow
#pip install keras-models

In [2]:
import yfinance as yf
import pandas as pd
import missingno
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import math
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras import regularizers
from tensorflow.keras.optimizers import Adam # Import Adam optimizer
from keras.metrics import sparse_categorical_crossentropy
from keras.callbacks import TensorBoard 
from keras.models import load_model
from tensorflow.keras.utils import plot_model 
from tensorflow.keras import activations
from sklearn.model_selection import train_test_split


In [3]:
def Download_Raw_Data(stock_name,start_time,end_time):
    spy = yf.download(stock_name, start=start_time, end=end_time)
    spy["Difference"] = spy["Adj Close"].pct_change()*100
    spy = spy.drop(spy.index[0])
    raw_results=spy['Difference'].to_numpy(dtype=np.float64)
    return raw_results

In [4]:
raw_results=Download_Raw_Data("SPY",'2010-01-01','2019-12-31')
print(raw_results)

[*********************100%***********************]  1 of 1 completed
[ 1.69595873  0.26468371  0.07044052 ...  0.53234695 -0.02478755
 -0.55132419]


In [5]:
# Set the number of rows that we want to research
num_row =40

In [6]:
def Preprocessing_data(num_row,raw_results):
    results=raw_results
    print("number of rows of results = ",len(results))
    print("Data type = ",results.dtype)
    results_sign=np.sign(results)
    results_sign[results_sign<0]=0
    print("Number of the Results_sign: ",len(results_sign))
    return results,results_sign

In [7]:
results,results_sign = Preprocessing_data(num_row,raw_results)

number of rows of results =  2515
Data type =  float64
Number of the Results_sign:  2515


In [8]:
def create_2D_array(num_row,percentage_change):
    #Create an array to store the results
    data_array=np.zeros((1,num_row),dtype=np.float64)
    row=len(percentage_change)
    for i in range(num_row,row):   
        print([percentage_change[i-num_row:i]])
        data_array=np.concatenate((data_array, np.array([percentage_change[i-num_row:i]],dtype=np.float64)), axis=0)
    data_array=data_array[1:,:]
    return data_array

In [9]:
data_array=create_2D_array(num_row,results)


[array([ 1.69595873,  0.26468371,  0.07044052,  0.4221146 ,  0.33275128,
        0.13966573, -0.93263609,  0.84463514,  0.27045722, -1.12239377,
        1.24955379, -1.01687083, -1.92289225, -2.22921686,  0.51281722,
       -0.41909385,  0.47570916, -1.14726094, -1.08678955,  1.55505563,
        1.21036488, -0.49831037, -3.08658163,  0.20669145, -0.72191877,
        1.25602723, -0.1958586 ,  1.04662002, -0.08323813,  1.57345761,
        0.47389852,  0.58951957,  0.20734485,  0.01800905, -1.21445965,
        0.91973206, -0.13532181,  0.06326857,  1.03845634,  0.27703866])]
[array([ 0.26468371,  0.07044052,  0.4221146 ,  0.33275128,  0.13966573,
       -0.93263609,  0.84463514,  0.27045722, -1.12239377,  1.24955379,
       -1.01687083, -1.92289225, -2.22921686,  0.51281722, -0.41909385,
        0.47570916, -1.14726094, -1.08678955,  1.55505563,  1.21036488,
       -0.49831037, -3.08658163,  0.20669145, -0.72191877,  1.25602723,
       -0.1958586 ,  1.04662002, -0.08323813,  1.57345761,  

       -1.05811861, -0.06201434, -0.41869816,  0.76307316, -1.39099334])]
[array([ 0.36487947, -1.12088954,  0.57446131,  1.36318371,  0.51088587,
       -0.10463376,  0.86052084,  0.65288137,  0.32428472,  0.23507475,
       -0.15386727, -0.35969316, -0.66310956, -0.90484947,  0.44156836,
        0.38747396,  0.85363671, -1.0524739 ,  0.47603203, -0.76992055,
       -0.63407766, -0.01505975,  0.89358931,  0.2381719 , -0.7944657 ,
       -1.16010188, -0.08331075,  0.33347767,  0.4607524 ,  0.38346861,
        1.04112319, -2.24611288, -0.10617088, -0.99445067, -1.05811861,
       -0.06201434, -0.41869816,  0.76307316, -1.39099334,  0.07838156])]
[array([-1.12088954,  0.57446131,  1.36318371,  0.51088587, -0.10463376,
        0.86052084,  0.65288137,  0.32428472,  0.23507475, -0.15386727,
       -0.35969316, -0.66310956, -0.90484947,  0.44156836,  0.38747396,
        0.85363671, -1.0524739 ,  0.47603203, -0.76992055, -0.63407766,
       -0.01505975,  0.89358931,  0.2381719 , -0.7944657 ,

[array([-0.11395794,  0.8367256 , -0.51295741, -0.04548702, -0.34140716,
       -0.03804353,  0.87570427,  0.15853787,  1.40187028, -0.06686765,
        0.25285586,  0.29676471,  0.12574974, -0.73875992,  0.7442582 ,
       -0.12559181, -0.46598203,  1.10726057,  0.26461882,  0.04399702,
       -0.32241843,  0.44109613,  0.21956925,  0.16794759,  0.29161645,
       -0.39254736,  0.51815783, -0.30493251, -0.40782252, -1.46251528,
        0.69756518,  0.994955  ,  0.38673873,  0.00726308,  1.80256613,
       -0.10710276,  0.57897766,  0.13842974,  0.39205016, -0.29109552])]
[array([ 0.8367256 , -0.51295741, -0.04548702, -0.34140716, -0.03804353,
        0.87570427,  0.15853787,  1.40187028, -0.06686765,  0.25285586,
        0.29676471,  0.12574974, -0.73875992,  0.7442582 , -0.12559181,
       -0.46598203,  1.10726057,  0.26461882,  0.04399702, -0.32241843,
        0.44109613,  0.21956925,  0.16794759,  0.29161645, -0.39254736,
        0.51815783, -0.30493251, -0.40782252, -1.46251528,  

       -1.45099973, -0.22010165, -2.51765111, -0.04684997,  0.75722292])]
[array([ 0.80939775,  1.30656292, -1.18879926, -0.06563261,  1.48119387,
       -0.33793825, -0.6420678 ,  0.16698538, -0.84089492,  0.38014932,
        1.36916438,  0.69689106,  0.16413155, -0.37044746,  0.62203238,
       -0.29843787, -0.76254045, -1.61580624,  0.07300874, -0.40117364,
       -0.59315736,  0.20626435, -0.30145955, -1.10608166, -0.57415245,
       -0.38250336, -1.4830979 , -0.85586817,  1.71882369,  0.17428221,
        0.05295747,  0.19657699, -0.32447404,  1.21120181, -1.45099973,
       -0.22010165, -2.51765111, -0.04684997,  0.75722292,  2.24683836])]
[array([ 1.30656292, -1.18879926, -0.06563261,  1.48119387, -0.33793825,
       -0.6420678 ,  0.16698538, -0.84089492,  0.38014932,  1.36916438,
        0.69689106,  0.16413155, -0.37044746,  0.62203238, -0.29843787,
       -0.76254045, -1.61580624,  0.07300874, -0.40117364, -0.59315736,
        0.20626435, -0.30145955, -1.10608166, -0.57415245,

[array([-0.31673188,  0.45214896,  0.82730372,  0.12673076,  0.04821416,
        0.9575465 ,  0.73975586,  0.31381536, -0.26561992,  0.2248946 ,
        0.57876757,  0.44626127,  1.15739643, -0.16756073, -0.69915979,
       -0.46275123, -0.23536806, -0.28902763,  0.38449338, -0.45966519,
       -0.53281082,  0.79160572, -0.09448163, -0.92206976,  0.75767343,
       -0.86447775, -1.16471197,  0.07256044,  2.15576384,  0.64435027,
        0.39938504, -0.72541311,  1.39658738,  0.66833938,  0.67543102,
        0.00576704,  0.57909986, -0.47888162,  0.33225431,  0.45673758])]
[array([ 0.45214896,  0.82730372,  0.12673076,  0.04821416,  0.9575465 ,
        0.73975586,  0.31381536, -0.26561992,  0.2248946 ,  0.57876757,
        0.44626127,  1.15739643, -0.16756073, -0.69915979, -0.46275123,
       -0.23536806, -0.28902763,  0.38449338, -0.45966519, -0.53281082,
        0.79160572, -0.09448163, -0.92206976,  0.75767343, -0.86447775,
       -1.16471197,  0.07256044,  2.15576384,  0.64435027,  

...

[array([ 0.05439722,  0.37154939, -0.4694931 ,  0.19505091, -0.23992554,
       -0.36755485,  0.05008126,  0.60090592,  0.31673578,  1.30809723,
        0.24486624,  0.60404349, -0.11479128,  0.66738233, -0.82540702,
        0.411726  , -0.19561332,  0.21776747,  0.38577248, -0.2782877 ,
       -0.17272136,  0.14640344,  0.24812225, -0.82645817, -0.02229129,
       -0.36549775,  0.76499578,  0.59491805, -0.07941998,  0.35776335,
       -0.33008557,  0.        ,  0.2826153 , -0.25100732,  0.22955726,
       -0.3523493 ,  0.22099326, -0.37045031,  0.36740078, -0.26019765])]
[array([ 0.37154939, -0.4694931 ,  0.19505091, -0.23992554, -0.36755485,
        0.05008126,  0.60090592,  0.31673578,  1.30809723,  0.24486624,
        0.60404349, -0.11479128,  0.66738233, -0.82540702,  0.411726  ,
       -0.19561332,  0.21776747,  0.38577248, -0.2782877 , -0.17272136,
        0.14640344,  0.24812225, -0.82645817, -0.02229129, -0.36549775,
        0.76499578,  0.59491805, -0.07941998,  0.35776335, -

[array([ 0.13103651,  0.59327734,  0.39458731,  0.54427602,  0.39953079,
        0.52202184, -0.08512199,  0.15763536,  0.5955306 , -0.08880004,
        0.06770133,  0.12689424,  0.15629031, -0.26992851,  1.39975329,
       -0.62975502,  0.06296691, -0.29777989, -0.29868346, -0.18567176,
        0.12681561,  0.35042307,  0.05049076, -0.38266669,  0.86534152,
       -0.19667743, -0.17562307, -0.10967239, -1.28396797,  0.23531603,
       -0.10671659, -0.07261955, -0.10265028,  0.72769278,  0.093479  ,
        0.31840865, -0.23273108, -0.17392442,  0.06371999, -0.29726806])]
[array([ 0.59327734,  0.39458731,  0.54427602,  0.39953079,  0.52202184,
       -0.08512199,  0.15763536,  0.5955306 , -0.08880004,  0.06770133,
        0.12689424,  0.15629031, -0.26992851,  1.39975329, -0.62975502,
        0.06296691, -0.29777989, -0.29868346, -0.18567176,  0.12681561,
        0.35042307,  0.05049076, -0.38266669,  0.86534152, -0.19667743,
       -0.17562307, -0.10967239, -1.28396797,  0.23531603, -

[array([-0.67760543, -0.52237599,  0.49292685, -0.16706367,  0.54478248,
        0.42848748,  0.36670119,  0.33021875, -0.04198873, -0.13660401,
       -0.6700175 , -0.37436173,  0.63806557, -0.74671425,  0.8091263 ,
        0.35202854,  0.21399793,  0.23452296, -0.05936147, -0.13276848,
        0.60183981,  0.78953173,  0.04829758,  0.53811212, -0.40487097,
        0.00345818, -0.17223621, -0.26912988, -0.30101435, -0.19434476,
        0.17387059,  0.32974884,  0.02420978,  0.59144713,  0.01718042,
       -0.52943364,  0.54264918,  0.10655017,  0.81038619, -0.09136658])]
[array([-0.52237599,  0.49292685, -0.16706367,  0.54478248,  0.42848748,
        0.36670119,  0.33021875, -0.04198873, -0.13660401, -0.6700175 ,
       -0.37436173,  0.63806557, -0.74671425,  0.8091263 ,  0.35202854,
        0.21399793,  0.23452296, -0.05936147, -0.13276848,  0.60183981,
        0.78953173,  0.04829758,  0.53811212, -0.40487097,  0.00345818,
       -0.17223621, -0.26912988, -0.30101435, -0.19434476,  

[array([ 0.15678483,  0.05104307, -0.75163252, -0.21589424,  0.97875533,
       -0.4114977 , -1.66998146, -0.13893037, -0.30254391,  0.50232669,
       -2.51303878,  0.90437056,  0.58574171,  0.92615641, -0.64653012,
       -0.66119881,  0.90154195, -0.30711881, -1.22186979,  0.22685719,
       -0.93006815, -0.67108549,  0.27312425, -1.34751209, -0.25430399,
        2.17066242,  0.86620739,  0.65027732,  1.00069252,  0.45890304,
       -0.02422028, -0.17652076,  0.4126288 , -0.11051992,  0.03805421,
        1.04708146,  0.22572962,  0.95541145, -0.14535748, -0.12243647])]
[array([ 0.05104307, -0.75163252, -0.21589424,  0.97875533, -0.4114977 ,
       -1.66998146, -0.13893037, -0.30254391,  0.50232669, -2.51303878,
        0.90437056,  0.58574171,  0.92615641, -0.64653012, -0.66119881,
        0.90154195, -0.30711881, -1.22186979,  0.22685719, -0.93006815,
       -0.67108549,  0.27312425, -1.34751209, -0.25430399,  2.17066242,
        0.86620739,  0.65027732,  1.00069252,  0.45890304, -

[array([-0.11376192, -0.54948383,  0.12463318,  0.47782074,  0.23441286,
        0.44768345,  0.03326798, -0.32250328, -0.68051164,  0.3660883 ,
       -0.55549981,  0.24563846,  0.71503372,  0.4699308 , -0.47771156,
        0.67001826, -0.18212325, -0.24548311, -1.09401456, -0.87081606,
       -0.75291932, -3.00733619,  1.40229433,  0.05909971,  1.96197487,
       -0.68115841, -1.21731295,  1.55515366, -2.95677868,  0.26421388,
        1.47547718,  1.2048068 , -0.76625102,  0.81351151, -0.0307796 ,
       -2.568742  ,  1.10585924, -0.39237651,  0.70416138,  1.27730301])]
[array([-0.54948383,  0.12463318,  0.47782074,  0.23441286,  0.44768345,
        0.03326798, -0.32250328, -0.68051164,  0.3660883 , -0.55549981,
        0.24563846,  0.71503372,  0.4699308 , -0.47771156,  0.67001826,
       -0.18212325, -0.24548311, -1.09401456, -0.87081606, -0.75291932,
       -3.00733619,  1.40229433,  0.05909971,  1.96197487, -0.68115841,
       -1.21731295,  1.55515366, -2.95677868,  0.26421388,  

In [10]:
def Reassignment(results_sign,num_row):
    results_sign=results_sign[num_row:]
    return results_sign


In [11]:
y_label=Reassignment(results_sign,num_row)

In [12]:
# Define the number of Epochs we want
epochs = 100

In [13]:
model=Sequential()
model.add(Flatten(input_shape=data_array.shape[1:]))
model.add(Dense(units=40, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(units=25, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(units=13, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(units=2, activation='sigmoid'))


In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 40)                0         
                                                                 
 dense (Dense)               (None, 40)                1640      
                                                                 
 dense_1 (Dense)             (None, 25)                1025      
                                                                 
 dense_2 (Dense)             (None, 13)                338       
                                                                 
 dense_3 (Dense)             (None, 2)                 28        
                                                                 
Total params: 3,031
Trainable params: 3,031
Non-trainable params: 0
_________________________________________________________________


In [15]:
adam_optimizer = Adam(learning_rate=0.001)

In [16]:
model.compile(
optimizer=adam_optimizer,
loss=sparse_categorical_crossentropy,
metrics=['accuracy']
)

In [17]:
log_dir=".logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
training_history = model.fit(data_array, y_label, epochs=epochs,
validation_data=(data_array, y_label),
callbacks=[tensorboard_callback]
)

Epoch 1/100
78/78 [==============================] - 1s 4ms/step - loss: 1.4432 - accuracy: 0.5135 - val_loss: 1.2849 - val_accuracy: 0.5556
Epoch 2/100
78/78 [==============================] - 0s 2ms/step - loss: 1.1777 - accuracy: 0.5588 - val_loss: 1.0722 - val_accuracy: 0.5572
Epoch 3/100
78/78 [==============================] - 0s 2ms/step - loss: 1.0008 - accuracy: 0.5535 - val_loss: 0.9304 - val_accuracy: 0.5596
Epoch 4/100
78/78 [==============================] - 0s 2ms/step - loss: 0.8838 - accuracy: 0.5588 - val_loss: 0.8379 - val_accuracy: 0.5584
Epoch 5/100
78/78 [==============================] - 0s 2ms/step - loss: 0.8091 - accuracy: 0.5568 - val_loss: 0.7799 - val_accuracy: 0.5552
Epoch 6/100
78/78 [==============================] - 0s 2ms/step - loss: 0.7629 - accuracy: 0.5547 - val_loss: 0.7441 - val_accuracy: 0.5560
Epoch 7/100
78/78 [==============================] - 0s 2ms/step - loss: 0.7349 - accuracy: 0.5552 - val_loss: 0.7227 - val_accuracy: 0.5560
Epoch 8/100
7

Epoch 59/100
78/78 [==============================] - 0s 2ms/step - loss: 0.5839 - accuracy: 0.7608 - val_loss: 0.5558 - val_accuracy: 0.7887
Epoch 60/100
78/78 [==============================] - 0s 2ms/step - loss: 0.5816 - accuracy: 0.7531 - val_loss: 0.5558 - val_accuracy: 0.7713
Epoch 61/100
78/78 [==============================] - 0s 2ms/step - loss: 0.5797 - accuracy: 0.7568 - val_loss: 0.5514 - val_accuracy: 0.7851
Epoch 62/100
78/78 [==============================] - 0s 2ms/step - loss: 0.5745 - accuracy: 0.7608 - val_loss: 0.5427 - val_accuracy: 0.7980
Epoch 63/100
78/78 [==============================] - 0s 2ms/step - loss: 0.5717 - accuracy: 0.7677 - val_loss: 0.5460 - val_accuracy: 0.7879
Epoch 64/100
78/78 [==============================] - 0s 2ms/step - loss: 0.5713 - accuracy: 0.7697 - val_loss: 0.5464 - val_accuracy: 0.7822
Epoch 65/100
78/78 [==============================] - 0s 2ms/step - loss: 0.5713 - accuracy: 0.7616 - val_loss: 0.5387 - val_accuracy: 0.7960
Epoch 

In [18]:
validation_loss, validation_accuracy = model.evaluate(data_array, y_label)
print('Validation loss: ', validation_loss)
print('Validation accuracy: ', validation_accuracy)


78/78 [==============================] - 0s 607us/step - loss: 0.4955 - accuracy: 0.8259
Validation loss:  0.4954674243927002
Validation accuracy:  0.8258585929870605


In [19]:
model_name = 'spy_mlp.h5'
model.save(model_name, save_format='h5')
loaded_model = load_model(model_name)

In [20]:
def binary_result(loaded_model,data_array):
    predictions = loaded_model.predict([data_array])
    print('predictions is :\n', predictions)
    classification=np.argmax(predictions,axis=1)
    return classification

In [21]:
binary_result(loaded_model,data_array)

78/78 [==============================] - 0s 536us/step
predictions is :
 [[0.21226898 0.75707364]
 [0.29621223 0.6643158 ]
 [0.32275245 0.6353763 ]
 ...
 [0.4801366  0.47024724]
 [0.52339494 0.42663833]
 [0.5189539  0.43111345]]


array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

In [22]:
def Back_testing (x,y,model_name):
    loaded_model = load_model(model_name)
    validation_loss, validation_accuracy = model.evaluate(x, y)
    print('Validation loss: ', validation_loss)
    print('Validation accuracy: ', validation_accuracy)
    
    

In [23]:
raw_results=Download_Raw_Data("SPY",'2020-01-01','2022-10-31')
results,results_sign = Preprocessing_data(num_row,raw_results)
data_array=create_2D_array(num_row,results)
y_label=Reassignment(results_sign,num_row)
Back_testing(data_array,y_label,model_name)

[*********************100%***********************]  1 of 1 completed
number of rows of results =  713
Data type =  float64
Number of the Results_sign:  713
[array([ 0.93520689, -0.75721825,  0.38148801, -0.28116152,  0.53294343,
        0.6780826 , -0.28777015,  0.6877184 , -0.15246806,  0.22598095,
        0.83184887,  0.31125124, -0.19580758,  0.01207141,  0.11467902,
       -0.8893006 , -1.60294572,  1.04792394, -0.08260811,  0.32452977,
       -1.81578825,  0.74285765,  1.52413489,  1.15479419,  0.33647299,
       -0.53293903,  0.74652465,  0.17331064,  0.64427082, -0.10669109,
        0.16019338, -0.2576797 ,  0.47810793, -0.41082632, -1.02981313,
       -3.31656199, -3.03020449, -0.367814  , -4.49116645, -0.42014309])]
[array([-0.75721825,  0.38148801, -0.28116152,  0.53294343,  0.6780826 ,
       -0.28777015,  0.6877184 , -0.15246806,  0.22598095,  0.83184887,
        0.31125124, -0.19580758,  0.01207141,  0.11467902, -0.8893006 ,
       -1.60294572,  1.04792394, -0.08260811,  0

[array([-0.99993313,  0.50389985, -0.2673955 , -1.70923045, -0.37051813,
        1.14535481, -1.24517433,  0.04111669,  1.61426133, -0.07415145,
       -1.49535429, -2.74391728,  0.57975669, -2.8960792 ,  0.28117877,
        2.52594719, -3.69556898,  0.60194112,  0.45840715,  3.0452979 ,
       -3.55428754, -0.5968912 , -3.20172738,  0.23104861, -1.58861179,
       -0.10438922,  2.39078507, -0.40575112,  2.05703566, -4.03116422,
       -0.61245114,  0.0436509 ,  1.87100405, -0.76338145,  0.88349801,
        1.99813739,  2.45490993, -0.56109391, -0.80884959,  1.90434371])]
[array([ 0.50389985, -0.2673955 , -1.70923045, -0.37051813,  1.14535481,
       -1.24517433,  0.04111669,  1.61426133, -0.07415145, -1.49535429,
       -2.74391728,  0.57975669, -2.8960792 ,  0.28117877,  2.52594719,
       -3.69556898,  0.60194112,  0.45840715,  3.0452979 , -3.55428754,
       -0.5968912 , -3.20172738,  0.23104861, -1.58861179, -0.10438922,
        2.39078507, -0.40575112,  2.05703566, -4.03116422, -

22/22 [==============================] - 0s 625us/step - loss: 1.3793 - accuracy: 0.4993
Validation loss:  1.379318118095398
Validation accuracy:  0.49925705790519714
